In [1]:
# Importing necessary libraries
import os
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import cv2

# Path to the input data and annotations
input_data_path = 'face-mask-detection/images'
annotations_path = "face-mask-detection/annotations"
images = [*os.listdir("face-mask-detection/images")]
output_data_path = '.'

In [2]:
def parse_annotation_object(annotation_object):
    params = {}
    for param in list(annotation_object):
        if param.tag == 'name':
            params['name'] = param.text
        if param.tag == 'bndbox':
            for coord in list(param):
                params[coord.tag] = int(coord.text)
    return params

In [3]:
dataset = []
for anno in glob.glob(annotations_path + "/*.xml"):
    tree = ET.parse(anno)
    root = tree.getroot()
    constants = {'file': root.find('filename').text[0:-4]}
    objects = root.findall('object')
    for obj in objects:
        object_params = parse_annotation_object(obj)
        dataset.append({**constants, **object_params})

df = pd.DataFrame(dataset)

In [4]:
final_test_image = 'maksssksksss0'
images.remove(f'{final_test_image}.png')
df = df[df["file"] != final_test_image]

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [6]:
for label in df['name'].unique():
    for d in ['train', 'test', 'val']:
        path = os.path.join(output_data_path, d, label)
        os.makedirs(path, exist_ok=True)

In [7]:
def crop_img(image_path, x_min, y_min, x_max, y_max):
    img = Image.open(image_path)
    cropped = img.crop((x_min - (x_max - x_min) * 0.1, y_min - (y_max - y_min) * 0.1, x_max + (x_max - x_min) * 0.1, y_max + (y_max - y_min) * 0.1))
    return cropped

# Saving images to directories
def save_image(image, image_name, dataset_type, label):
    output_path = os.path.join(output_data_path, dataset_type, label, f'{image_name}.png')
    image.save(output_path)

for dataset, dataset_type in [(train_df, 'train'), (test_df, 'test'), (val_df, 'val')]:
    for _, row in dataset.iterrows():
        image_path = os.path.join(input_data_path, row['file'] + '.png')
        image = crop_img(image_path, row['xmin'], row['ymin'], row['xmax'], row['ymax'])
        save_image(image, row['file'] + '_' + str((row['xmin'], row['ymin'])), dataset_type, row['name'])

In [8]:
model = Sequential([
    Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(35, 35, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(units=500, activation='relu'),
    Dropout(0.3),
    Dense(units=3, activation='softmax')
])

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [10]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, zoom_range=0.1, shear_range=0.2, width_shift_range=0.1,
    height_shift_range=0.1, rotation_range=4, vertical_flip=False
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [14]:
batch_size = 8
train_generator = datagen.flow_from_directory(directory='train', target_size=(35, 35),
                                              class_mode="categorical", batch_size=batch_size, shuffle=True)

val_generator = val_datagen.flow_from_directory(directory='val', target_size=(35, 35),
                                                class_mode="categorical", batch_size=batch_size, shuffle=True)

test_generator = val_datagen.flow_from_directory(directory='test', target_size=(35, 35),
                                                 class_mode="categorical", batch_size=batch_size, shuffle=False)

Found 22 images belonging to 3 classes.
Found 6 images belonging to 3 classes.
Found 7 images belonging to 3 classes.


In [15]:
model.fit_generator(generator=train_generator, epochs=50, validation_data=val_generator, callbacks=[])

Epoch 1/50


<ipython-input-15-60136a4efd23>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, epochs=50, validation_data=val_generator, callbacks=[])


3/3 [==============================] - 2s 152ms/step - loss: 0.7656 - accuracy: 0.5909 - val_loss: 2.0289 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 0s 95ms/step - loss: 0.3637 - accuracy: 0.9091 - val_loss: 2.6763 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 0s 90ms/step - loss: 0.4417 - accuracy: 0.9091 - val_loss: 2.5236 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 0s 42ms/step - loss: 0.3827 - accuracy: 0.9091 - val_loss: 1.6586 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 0s 48ms/step - loss: 0.2991 - accuracy: 0.9091 - val_loss: 1.3898 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - 0s 39ms/step - loss: 0.2944 - accuracy: 0.9091 - val_loss: 1.4080 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 0s 44ms/step - loss: 0.2679 - accuracy: 0.9091 - val_loss: 1.6079 - val_accuracy: 0.5000
Epoch 8/50
3/3 [=====================

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras import layers, models, datasets ,utils
from keras.applications import ResNet50, MobileNetV3Small, InceptionV3, VGG16
from keras.models import Model



# Load ResNet50 model pre-trained on ImageNet data and remove the top layer
base_model = VGG16(weights='imagenet', include_top=False)
base_model.trainable = False  # Freeze the model

# Adding global average pooling to convert features to vectors
global_average_layer = layers.GlobalAveragePooling2D()
feature_extractor = Model(inputs=base_model.input, outputs=global_average_layer(base_model.output))


58889256/58889256 [==============================] - 0s 0us/step


In [28]:
# Load ResNet50 model pre-trained on ImageNet data and remove the top layer
base_model = VGG16(input_shape=(35, 35, 3), include_top=False)
base_model.trainable = False  # Freeze the model

model_res = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128,activation='relu'),
    layers.Dense(3, activation='softmax')
])
model_res.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model_res.fit_generator(generator=train_generator, epochs=50, validation_data=val_generator, callbacks=[])

Epoch 1/50


<ipython-input-29-6898bd01c174>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_res.fit_generator(generator=train_generator, epochs=50, validation_data=val_generator, callbacks=[])


3/3 [==============================] - 2s 477ms/step - loss: 0.8864 - accuracy: 0.6364 - val_loss: 1.4390 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 1s 244ms/step - loss: 0.3639 - accuracy: 0.9091 - val_loss: 1.9340 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 0s 160ms/step - loss: 0.4435 - accuracy: 0.9091 - val_loss: 2.3613 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 0s 176ms/step - loss: 0.4270 - accuracy: 0.9091 - val_loss: 2.5950 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 0s 175ms/step - loss: 0.3365 - accuracy: 0.9091 - val_loss: 2.6985 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - ETA: 0s - loss: 0.3222 - accuracy: 0.9091

KeyboardInterrupt: 

In [ ]:
import keras
# Unfreeze the entire model for fine-tuning
base_model.trainable = True

# It's important to recompile the model after making any changes to the `trainable` attribute of any layer.
model_res.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [16]:
model_loss, model_acc = model.evaluate(test_generator)
print(f'Test Loss: {model_loss}, Test Accuracy: {model_acc}')

1/1 [==============================] - 0s 72ms/step - loss: 1.3983 - accuracy: 0.7143
Test Loss: 1.3983440399169922, Test Accuracy: 0.7142857313156128
